# Package Import

In [1]:
import itertools
import random
import subprocess
import os

# Function

In [2]:
def asign_k(k,generate):## inseart K to the generate part and return updated generate
    gen_ls=generate.splitlines()
    k_str='#const n='+str(k)+'.'
    gen_ls.insert(1, k_str)
    gen_update='\n'.join(gen_ls)
    return(gen_update)

In [3]:
def str_to_lp(string,filename,problem):
    if not os.path.exists("clingo_files"):
        os.makedirs("clingo_files")
    with open("clingo_files"+"/"+problem+"_"+filename+".lp", "w") as text_file:
        text_file.write(string)

In [4]:
def run_clingo(instance, gen, check,term):
    str_to_lp(instance,"instance",term)
    str_to_lp(gen,"generate",term)
    str_to_lp(check,"check",term)
    command='bash -c "clingo clingo_files/{}_instance.lp clingo_files/{}_generate.lp  clingo_files/{}_check.lp"'.format(term,term,term)
    result=run_bash(command)
    return(result)

In [5]:
def run_bash(command):
    stdout = subprocess.getoutput(command)
    result=stdout.split("\n")[-6]
    return(result)

# Graph

<img src="img/col_instance.png"  width="200"> 

Take the above graph as an example, we can translate it into clingo script

In [6]:
instance=""" 
e(1,2). 
e(2,3). 
e(3,4). 
e(4,1). 
e(4,5).
"""

# Coloring Generate

Each node will be assigned one of the k colors (see the figure below k=3)

<img src="img/col_generate.png" width="200"/> 

In [7]:
generate="""
gnode(X) :- e(X,_).
gnode(X) :- e(_,X).

{ col(X,1..n) } = 1 :- gnode(X).

"""

# Solution Check

In [8]:
check="""

:- e(X,Y), col(X,C), col(Y,C), X != Y.
#show col/2.

"""

# Problem1: K_COL

 Given a constant K, find all possible solutions to CAP with k containers

In [9]:
k=3

In [10]:
generate_update=asign_k(k,generate) # update generate with k colors

In [11]:
run_clingo(instance, generate_update, check,"k_col")

'SATISFIABLE'

# Problem2: min_K_COL

In [12]:
k=0
clingo_output=""
#find the minimum number of containers to satisfy all
while clingo_output!="SATISFIABLE":
    k=k+1  ## for other problems, the variables should be revised
    print("col number: "+str(k)) 
    generate_update=asign_k(k,generate) # update generate with k boxes
    clingo_output=run_clingo(instance, generate_update, check,"k_col")
    print(clingo_output)
print("\n\nMinimum K for this graph is: {}".format(k))

col number: 1
UNSATISFIABLE
col number: 2
SATISFIABLE


Minimum K for this graph is: 2
